In [ ]:
import pandas as pd

# Specify the path to your Excel file
file_path = r"09_Outputs\main_data.xlsx"

# Load the Excel file into a DataFrame using the openpyxl engine
df_main = pd.read_excel(file_path, engine='openpyxl')

# Remove quotation marks from the 'Entity' column
df_main['Entity'] = df_main['Entity'].str.replace('"', '')

# Retrieve unique countries from the DataFrame and store them as a list of strings
all_countries = df_main['Entity'].drop_duplicates().astype(str).tolist()

# Display the first 5 rows of the DataFrame for a quick overview
df_main.head()


In [ ]:
# Example Filter the DataFrame for Germany
germany_df = df_main[df_main['Entity'] == "Germany"]

# Display the DataFrame for Germany
germany_df.head()



In [ ]:
## Easy Testplot
import pandas as pd
import matplotlib.pyplot as plt

# Filter the DataFrame for the desired countries
countries = ['Germany', 'China', 'India', 'United States', 'World']
filtered_df = df_main[df_main.iloc[:, 0].isin(countries)]

# Create a plot
plt.figure(figsize=(12, 8))

# Plot the data for each country
for country in countries:
    country_data = filtered_df[filtered_df.iloc[:, 0] == country]
    plt.plot(country_data.iloc[:, 1], country_data.iloc[:, 3], label=country)

plt.title('Annual CO₂ Emissions')
plt.xlabel('Year')
plt.ylabel('Annual CO₂ Emissions')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
## Zeige deskriptive Statistiken
#print(df_main.describe())
#
## Zeige die ersten Zeilen des DataFrames
#print(df_main.head())
#
## Zeige die Datentypen jeder Spalte
#print(df_main.dtypes)
#

In [ ]:
## Sample

subset = df_main.iloc[:5, :5]
display(subset)



In [ ]:
print(all_countries[:7])


In [ ]:
df_main.dtypes


In [ ]:
print(df_main.columns)


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import HTML



# Presuming df_main is already available
filtered_df = df_main

# Extract unique countries from the dataframe
all_countries = sorted(filtered_df['Entity'].drop_duplicates().astype(str).tolist())

# Set a default list of countries for the user interface
default_countries = ['Germany', 'China', 'India', 'United States', 'World']

# Update list of countries based on user search input
def update_country_list(*args):
    # Extracting selected countries based on current checkbox selections
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]

    search_input = search_box.value
    # Filtering countries based on search criteria
    search_results = [country for country in all_countries if search_input.lower() in country.lower()]
    
    # Merge the search results with previously selected countries and remove duplicates
    updated_countries = list(set(search_results + selected_countries))
    
    # Prioritize selected countries to be shown at the top
    updated_countries = sorted(updated_countries, key=lambda x: (x not in selected_countries, x))
    
    updated_checkboxes = [widgets.Checkbox(value=(country in selected_countries), description=country,
                                           layout=widgets.Layout(width="100%", padding="0px")) for country in updated_countries]
    for cb in updated_checkboxes:
        cb.observe(plot_selected_countries, names='value')
    checkboxes_container.children = tuple(updated_checkboxes)


# Creating the search box widget
search_box = widgets.Text(value='', placeholder='Search for Countries...', description='', layout=widgets.Layout(width="95%"))
search_box.observe(update_country_list, names='value')

clear_button = widgets.Button(description="X", layout=widgets.Layout(width="5%"))
def clear_search_field(change):
    search_box.value = ''
clear_button.on_click(clear_search_field)

# Creating country checkboxes widget
country_checkboxes = [widgets.Checkbox(value=(country in default_countries), description=country,
                                       layout=widgets.Layout(width="100%", padding="0px")) for country in all_countries]
checkboxes_container = widgets.VBox(country_checkboxes,
                                    layout=widgets.Layout(overflow_y="scroll", height="400px", border="1px solid #DDD"))

# Output widget to display the plot
out = widgets.Output(layout=widgets.Layout(height="400px", width="60%"))
out.layout.height = "400px"
checkboxes_container.layout.height = "400px"

# Function to plot data for the selected countries
def plot_selected_countries(change=None):
    # Extracting selected countries based on user checkbox selections
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    
    # Sort checkboxes to place selected countries on top
    sorted_checkboxes = sorted(checkboxes_container.children, key=lambda cb: (cb.description not in selected_countries, cb.description))
    checkboxes_container.children = tuple(sorted_checkboxes)
    
    with out:
        clear_output(wait=True)
        fig = go.Figure()
        fig_height = 400  
        fig.update_layout(height=fig_height)
        for country in selected_countries:
            country_data = filtered_df[filtered_df['Entity'] == country]
            # Adding trace for each country. The mode 'lines+markers' will display markers at data points.
            fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data['Annual CO₂ emissions in tons'], mode='lines', name=country, showlegend=True))

        # Updating layout for the plot with grid and customizing appearance
        fig.update_layout(title='Annual CO₂ Emissions in tonnes', 
                          xaxis=dict(title='Year', showgrid=True, gridcolor='rgba(128, 128, 128, 0.5)', gridwidth=0.5, griddash='dot', zeroline=False),
                          yaxis=dict(title='Annual CO₂ Emissions', showgrid=True, gridcolor='rgba(128, 128, 128, 0.5)', gridwidth=0.5, griddash='dot', zeroline=False),
                          showlegend=True, xaxis_title='', 
                          plot_bgcolor='white', 
                          paper_bgcolor='white')
        font_size_px = 16
        fig.update_layout(
            font=dict(size=font_size_px),
            title=dict(font=dict(size=font_size_px)),
            xaxis=dict(titlefont=dict(size=font_size_px), tickfont=dict(size=font_size_px)),
            yaxis=dict(titlefont=dict(size=font_size_px), tickfont=dict(size=font_size_px)),
            legend=dict(font=dict(size=font_size_px))
        )

        fig.show()

# Adding listener for change in checkbox selection to update the plot
for cb in country_checkboxes:
    cb.observe(plot_selected_countries, names='value')

left_container_width = '270px'  # Zum Beispiel, wenn die ursprüngliche Breite 200px war
left_container = widgets.VBox([widgets.HBox([search_box, clear_button]), checkboxes_container],
                              layout=widgets.Layout(width=left_container_width, border="1px solid #DDD", padding="5px 0px 0px 0px"))
    
display(HTML("""
<style>
    :root {
        --jp-widgets-inline-label-width: auto !important;
    }
</style>
"""))

display(widgets.HBox([left_container, out]))
plot_selected_countries()


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import HTML




# Presuming df_main is already available
filtered_df = df_main

# Extract unique countries from the dataframe
all_countries = sorted(filtered_df['Entity'].drop_duplicates().astype(str).tolist())

# Set a default list of countries for the user interface
default_countries = ['Germany', 'China', 'India', 'United States', 'World']
left_container_width = '270px'  # Zum Beispiel, wenn die ursprüngliche Breite 200px war


# Update list of countries based on user search input
def update_country_list(*args):
    # Extracting selected countries based on current checkbox selections
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]

    search_input = search_box.value
    # Filtering countries based on search criteria
    search_results = [country for country in all_countries if search_input.lower() in country.lower()]
    
    # Merge the search results with previously selected countries and remove duplicates
    updated_countries = list(set(search_results + selected_countries))
    
    # Prioritize selected countries to be shown at the top
    updated_countries = sorted(updated_countries, key=lambda x: (x not in selected_countries, x))
    
    updated_checkboxes = [widgets.Checkbox(value=(country in selected_countries), description=country,
                                           layout=widgets.Layout(width="100%", padding="0px")) for country in updated_countries]
    for cb in updated_checkboxes:
        cb.observe(plot_selected_countries, names='value')
    checkboxes_container.children = tuple(updated_checkboxes)


# Creating the search box widget
search_box = widgets.Text(value='', placeholder='Search for Countries...', description='', layout=widgets.Layout(width="95%"))
search_box.observe(update_country_list, names='value')

clear_button = widgets.Button(description="X", layout=widgets.Layout(width="5%"))
def clear_search_field(change):
    search_box.value = ''
clear_button.on_click(clear_search_field)

# Creating country checkboxes widget
country_checkboxes = [widgets.Checkbox(value=(country in default_countries), description=country,
                                       layout=widgets.Layout(width="100%", padding="0px")) for country in all_countries]
checkboxes_container = widgets.VBox(country_checkboxes,
                                    layout=widgets.Layout(overflow_y="scroll", height="400px", border="1px solid #DDD"))

# Output widget to display the plot
out = widgets.Output(layout=widgets.Layout(height="400px", width="60%"))
out.layout.height = "400px"
checkboxes_container.layout.height = "400px"

# Function to plot data for the selected countries
def create_plot_for_column(column):
    
    # Funktion, um Daten für die ausgewählten Länder zu plotten
    def plot_for_column(change=None):
        selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
        sorted_checkboxes = sorted(checkboxes_container.children, key=lambda cb: (cb.description not in selected_countries, cb.description))
        checkboxes_container.children = tuple(sorted_checkboxes)

        with out:
            clear_output(wait=True)
            fig = go.Figure()
            fig_height = 400  
            fig.update_layout(height=fig_height)
            for country in selected_countries:
                country_data = filtered_df[filtered_df['Entity'] == country]
                fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data[column], mode='lines', name=country, showlegend=True))
            
            # Aktualisieren des Layouts für den Plot
            fig.update_layout(title=column, 
                              xaxis_title='Year',
                              yaxis_title=column,
                              plot_bgcolor='white', 
                              paper_bgcolor='white')

            # ... (restliche Plot-Einstellungen)

            fig.show()
    
    # Erstellen von Widgets speziell für diese Spalte
    country_checkboxes = [widgets.Checkbox(value=(country in default_countries), description=country,
                                           layout=widgets.Layout(width="100%", padding="0px")) for country in all_countries]
    for cb in country_checkboxes:
        cb.observe(plot_for_column, names='value')
    checkboxes_container = widgets.VBox(country_checkboxes, layout=widgets.Layout(overflow_y="scroll", height="400px", border="1px solid #DDD"))
    
    out = widgets.Output(layout=widgets.Layout(height="400px", width="60%"))

    left_container = widgets.VBox([widgets.HBox([search_box, clear_button]), checkboxes_container],
                                  layout=widgets.Layout(width=left_container_width, border="1px solid #DDD", padding="5px 0px 0px 0px"))

    # Display the output for this column
    display(widgets.VBox([widgets.HBox([left_container, out])]))
    plot_for_column()

# Für jede Spalte, die Sie plotten möchten:
columns_to_plot = filtered_df.columns[df_main.columns.get_loc('Code')+1:]
for col in columns_to_plot:
    create_plot_for_column(col)


# Adding listener for change in checkbox selection to update the plot
for cb in country_checkboxes:
    cb.observe(plot_selected_countries, names='value')


left_container = widgets.VBox([widgets.HBox([search_box, clear_button]), checkboxes_container],
                              layout=widgets.Layout(width=left_container_width, border="1px solid #DDD", padding="5px 0px 0px 0px"))
    
display(HTML("""
<style>
    :root {
        --jp-widgets-inline-label-width: auto !important;
    }
</style>
"""))

display(widgets.HBox([left_container, out]))
plot_selected_countries()


In [ ]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.core.display import HTML

# Presuming df_main is already available
filtered_df = df_main

# Extract unique countries from the dataframe
all_countries = sorted(filtered_df['Entity'].drop_duplicates().astype(str).tolist())

# Set a default list of countries for the user interface
default_countries = ['Germany', 'China', 'India', 'United States', 'World']
left_container_width = '270px'

# Update list of countries based on user search input
def update_country_list(*args):
    selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
    search_input = search_box.value
    search_results = [country for country in all_countries if search_input.lower() in country.lower()]
    updated_countries = list(set(search_results + selected_countries))
    updated_countries = sorted(updated_countries, key=lambda x: (x not in selected_countries, x))
    updated_checkboxes = [widgets.Checkbox(value=(country in selected_countries), description=country,
                                           layout=widgets.Layout(width="100%", padding="0px")) for country in updated_countries]
    checkboxes_container.children = tuple(updated_checkboxes)

# Creating the search box widget
search_box = widgets.Text(value='', placeholder='Search for Countries...', description='', layout=widgets.Layout(width="95%"))
search_box.observe(update_country_list, names='value')

clear_button = widgets.Button(description="X", layout=widgets.Layout(width="5%"))
def clear_search_field(change):
    search_box.value = ''
clear_button.on_click(clear_search_field)

# Creating country checkboxes widget
country_checkboxes = [widgets.Checkbox(value=(country in default_countries), description=country,
                                       layout=widgets.Layout(width="100%", padding="0px")) for country in all_countries]
checkboxes_container = widgets.VBox(country_checkboxes, layout=widgets.Layout(overflow_y="scroll", height="400px", border="1px solid #DDD"))

# Output widget to display the plot
out = widgets.Output(layout=widgets.Layout(height="400px", width="60%"))

# Function to plot data for the selected countries
def create_plot_for_column(column):
    def plot_for_column(change=None):
        selected_countries = [cb.description for cb in checkboxes_container.children if cb.value]
        with out:
            clear_output(wait=True)
            fig = go.Figure()
            fig_height = 400  
            fig.update_layout(height=fig_height)
            for country in selected_countries:
                country_data = filtered_df[filtered_df['Entity'] == country]
                fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data[column], mode='lines', name=country, showlegend=True))
            fig.update_layout(title=column, xaxis_title='Year', yaxis_title=column, plot_bgcolor='white', paper_bgcolor='white')
            fig.show()

    display(widgets.VBox([widgets.HBox([left_container, out])]))
    plot_for_column()

# Für jede Spalte, die Sie plotten möchten:
columns_to_plot = filtered_df.columns[filtered_df.columns.get_loc('Code')+1:]
for col in columns_to_plot:
    create_plot_for_column(col)

left_container = widgets.VBox([widgets.HBox([search_box, clear_button]), checkboxes_container],
                              layout=widgets.Layout(width=left_container_width, border="1px solid #DDD", padding="5px 0px 0px 0px"))

display(HTML("""
<style>
    :root {
        --jp-widgets-inline-label-width: auto !important;
    }
</style>
"""))

display(widgets.HBox([left_container, out]))
